In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np # math
import matplotlib.pyplot as plt # graphs and plots
import pandas as pd
from tqdm import tqdm
import datetime
import matplotlib.dates as mdates
from src.utils.profiles_from_schedule_new import SimParam # class for show-up generator
from src.simfunc.KIX_T1d_CUSBD_new import Pax, Simulation , minutes_to_hms# classes for simulation
import seaborn as sns


In [ ]:
simparam = SimParam()
path_to_6k_sched = (
    r"/home/antoine/projects/"
    "KAPpaxsim/KAPpaxsim/data/"
    "processed/Schedule (30th terminal peak, 6000 pax)_PROCESSED.xlsx"
)
simparam.schedule_from_path(path_to_6k_sched)
simparam.schedule_cleanup()
simparam.schedule_filter()
simparam.show_up_from_file()
simparam.assign_flight_show_up_category_default()
# simparam.plot_show_up_categories_profiles()
simparam.schedule['PAX_SUM FC'] = 10
simparam.assign_show_up()
# simparam.plot_df_Pax()
simparam.assign_check_in()

In [ ]:
# simparam.df_Counters['total'].plot()

In [ ]:
simparam.dct_resource = {
    'kiosk' : 92,
    'security' : 200,
    'CUSBD' : 192,
}

simparam.dct_processes = {
    'kiosk' : 70,
    'security': 1,
    'CUSBD': 60,
    'checkin': 70,
}

In [ ]:
simulation = Simulation(simparam)
simulation.generate_checkin()
simulation.generate_pax()
simulation.run(end_time=5000)

In [ ]:
simulation.format_df_result()
simulation.plot_result()

In [ ]:
df_result = pd.concat(
            [pax.row for pax in simulation.pax_list], axis=1
        ).transpose()

FREQ = "15min"
WINDOW = 4

In [ ]:
# sampling ratio
ratio_sampling = pd.to_timedelta("1H") / pd.to_timedelta(FREQ)

# list for iteration
list_process_all = ["kiosk", "checkin"]

# different types of columns
datetime_columns_types = [
    "start_{}_queue",
    "end_{}_queue",
    "end_{}_process",
]

datetime_columns = [
    a.format(b) for b in list_process_all for a in datetime_columns_types
]

# change datetime columns to datetime
for column in datetime_columns:
    df_result[column] = pd.to_datetime(
        df_result[column].apply(lambda x: minutes_to_hms(x))
    )

# calculate waiting times
for process in list_process_all:
    df_result["wait_time_{}".format(process)] = (
        df_result["end_{}_queue".format(process)]
        - df_result["start_{}_queue".format(process)]
    ).fillna(datetime.timedelta(0))

# artificial high waiting time for pax who did not finish
for process in list_process_all:
    mask = (pd.isna(df_result["end_{}_queue".format(process)])) & (
        pd.notna(df_result["start_{}_queue".format(process)])
    )

    df_result.loc[
        mask, "wait_time_{}".format(process)
    ] = datetime.timedelta(hours=8)

# aggregates for plotting
list_plot = [
    "start_{}_queue",
    "end_{}_process",
    "{}_queue_length",
    "wait_time_{}",
]
dct_plot = {
    key: [plot.format(key) for plot in list_plot] for key in list_process_all
}

# in
plt_in = [
    (
        df_result.set_index(dct_plot[key][0], drop=False)["Pax"]
        .resample(FREQ)
        .agg(["sum"])
        .rolling(window=WINDOW, center=True)
        .mean()
        .dropna()
        .apply(lambda x: x * ratio_sampling)
    )
    for key in [*dct_plot]
]

# out
plt_out = [
    (
        df_result.set_index(dct_plot[key][1], drop=False)["Pax"]
        .resample(FREQ)
        .agg(["sum"])
        .rolling(window=WINDOW, center=True)
        .mean()
        .dropna()
        .apply(lambda x: x * ratio_sampling)
    )
    for key in [*dct_plot]
]



In [ ]:
# queue length
plt_queue_length = [
    (
        df_result.set_index(dct_plot[key][0], drop=False)[
            dct_plot[key][2]
        ]
        .dropna()
        .resample(FREQ)
        .agg(["max"])
        .rolling(window=WINDOW, center=True)
        .mean()
    )
    for key in [*dct_plot]
]

# queue duration
plt_queue_duration = [
    (
        df_result.set_index(dct_plot[key][0], drop=False)[
            dct_plot[key][3]
        ]
        .apply(lambda x: x.total_seconds() / 60)
        .resample(FREQ)
        .agg(["max"])
        .rolling(window=WINDOW, center=True)
        .mean()
    )
    for key in [*dct_plot]
]

# histograms of queue duration and queue length
dct_hist_wait_time = {
    key: (
        df_result[df_result[dct_plot[key][0]].notnull()][
            dct_plot[key][3]
        ].apply(lambda x: x.total_seconds() / 60)
    )
    for key in [*dct_plot]
}

dct_hist_queue_length = {
    key: (
        df_result[df_result[dct_plot[key][0]].notnull()][
            dct_plot[key][2]
        ]
    )
    for key in [*dct_plot]
}

plt_hist_wait_time = [value for value in dct_hist_wait_time.values()]


In [ ]:
dct_plot

In [ ]:
df_result.set_index(dct_plot['checkin'][0], drop=False)[dct_plot['checkin'][2]]

In [ ]:
df_result

In [ ]:
simulation.pax_list[1500].row

In [ ]:
simulation.df_result['time'][0].hour

In [ ]:
simulation.df_result[simulation.df_result['time'].apply(lambda x : x.hour) == 16]

In [ ]:
len(simulation.checkin['DKH'].queue)